In [1]:
# importing the librtaries
import math
import numpy as np
import nltk
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer 
import heapq

In [2]:
import pandas as pd

In [7]:
with open('TrainData.txt') as f:
    lines = f.readlines()

In [8]:
lines

['Answer\tQuestion\n',
 "Yes. It's absolutely beautiful today.\tThe weather is great isn't it?\n",
 "yes, i like that one, too.\tthat one. the one that's all black.\n",
 "it's really nice.\ti got it from macy's.\n",
 'at 8:00 p.m.\twhen does it start?\n',
 "nothing, except my favorite color is blue.\twhat's the matter with green eyes?\n",
 ' I have read just about everything in Project Gutenberg.\t have you ever read a book\n',
 ' a fancy name for applied computer science in biology.\t what is bioinformatics\n',
 'Sure, the changing rooms are over there.\tCan I try it on?\n',
 'I want it taken from my checking account.\tWhich account are you making this withdrawal from?\n',
 "and you get to play with a lot of dogs.\tthat's the truth.\n",
 ' A game with tall players.\t WHAT IS BASKETBALL\n',
 'and then we can go to my home.\twe can watch my dvd.\n',
 'i will talk to your father about that.\ti saw dad wipe his nose on his sleeve yesterday.\n',
 'Thank you so much.\tI can cancel it for yo

In [23]:
questions = []
answers = []

In [24]:
for l in lines:
    as_list = l.split("\t")
    questions.append(as_list[1].replace("\n",""))
    answers.append(as_list[0])

In [25]:
questions

['Question',
 "The weather is great isn't it?",
 "that one. the one that's all black.",
 "i got it from macy's.",
 'when does it start?',
 "what's the matter with green eyes?",
 ' have you ever read a book',
 ' what is bioinformatics',
 'Can I try it on?',
 'Which account are you making this withdrawal from?',
 "that's the truth.",
 ' WHAT IS BASKETBALL',
 'we can watch my dvd.',
 'i saw dad wipe his nose on his sleeve yesterday.',
 'I can cancel it for you right now.',
 "i'm sure everything will be okay in a day or two.",
 "i've actually been busy lately.",
 "i wish i was free that night. i'm kind of mad that i didn't go.",
 'How are you doing today?',
 "i'm going to the movies with a friend. how about you?",
 'what are they?',
 ' do you drink',
 'the price of stamps goes up and up.',
 'the forecast says that it will be warm on the weekend.',
 'How long is this bus ride?',
 'How was the movie?',
 'i mixed them together.',
 'oh, really? maybe you should have called 911.',
 "we'll get w

In [26]:
questions = questions[1:]

In [28]:
questions[1:5]

["that one. the one that's all black.",
 "i got it from macy's.",
 'when does it start?',
 "what's the matter with green eyes?"]

In [29]:
answers

['Answer',
 "Yes. It's absolutely beautiful today.",
 'yes, i like that one, too.',
 "it's really nice.",
 'at 8:00 p.m.',
 'nothing, except my favorite color is blue.',
 ' I have read just about everything in Project Gutenberg.',
 ' a fancy name for applied computer science in biology.',
 'Sure, the changing rooms are over there.',
 'I want it taken from my checking account.',
 'and you get to play with a lot of dogs.',
 ' A game with tall players.',
 'and then we can go to my home.',
 'i will talk to your father about that.',
 'Thank you so much.',
 "it's your girlfriend's fault. she should have warned you.",
 'what have you been doing?',
 'it was a great game.',
 'I am fine',
 "i'm not sure yet.",
 'they are bites.',
 ' I am not capable of doing so.',
 'i think stamps used to cost a penny.',
 "so do you think it'll be perfect weather for the beach?",
 'It only takes half an hour.',
 'It was very interesting. You should watch it.',
 "if the coffee still tastes okay, that's a good ide

In [30]:
answers = answers[1:]

In [32]:
answers[0:5]

["Yes. It's absolutely beautiful today.",
 'yes, i like that one, too.',
 "it's really nice.",
 'at 8:00 p.m.',
 'nothing, except my favorite color is blue.']

In [37]:
len(questions)

5495

In [38]:
len(answers)

5495

In [40]:
df = pd.DataFrame({'Questions':questions,'Answers':answers})
df.head()

,Questions,Answers
0,The weather is great isn't it?,Yes. It's absolutely beautiful today.
1,that one. the one that's all black.,"yes, i like that one, too."
2,i got it from macy's.,it's really nice.
3,when does it start?,at 8:00 p.m.
4,what's the matter with green eyes?,"nothing, except my favorite color is blue."


In [82]:
df.to_csv('df.csv')

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495 entries, 0 to 5494
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Questions  5495 non-null   object
 1   Answers    5495 non-null   object
dtypes: object(2)
memory usage: 86.0+ KB


In [42]:
    !pip install Levenshtein

In [43]:
def getResults(questions, func):
    def getResult(q):
        answer, score, prediction = func(q)
        return [q, prediction, answer, score]
    return pd.DataFrame(list(map(getResult, questions)), columns=["Question", "Prediction", "Answer", "Score"])

In [44]:
test_df = pd.read_csv('TestData.csv')
test_df.head()

,I.D.,Question
0,QN_1,"i'll give you a speech like that, too."
1,QN_2,"i know, you're absolutely right."
2,QN_3,i liked it.
3,QN_4,the baby was eight pounds six ounces.
4,QN_5,I was sold a wireless service unavailable in m...


In [45]:
test_data = test_df['Question']

In [46]:
from Levenshtein import ratio

In [51]:
def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in df.iterrows():
        score = ratio(row["Questions"], q)
        if score >= 0.9:
            return row["Answers"], score, row["Questions"]
        elif score > max_score:
            max_score = score
            answer = row["Answers"]
            prediction = row["Questions"]
    if max_score > 0.3:
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction


In [53]:
predictions = getResults(test_data, getApproximateAnswer)

In [54]:
predictions

,Question,Prediction,Answer,Score
0,"i'll give you a speech like that, too.","i'll give you a speech like that, too.",do you think anyone will come to my funeral?,1.000000
1,"i know, you're absolutely right.","i know, you're absolutely right.",i wish it would cool off one day.,1.000000
2,i liked it.,i liked it.,"i'll give you a speech like that, too.",1.000000
3,the baby was eight pounds six ounces.,the baby was 8 pounds 6 ounces.,that's good to hear.,0.852941
4,I was sold a wireless service unavailable in m...,I was sold a wireless service unavailable in m...,"I see it here,we charged you $5 extra a month.",1.000000
...,...,...,...,...
538,at night i went out to eat. the food was delic...,at night i went out to eat. the food was delic...,people who live in hawaii are lucky.,1.000000
539,TELL ME ABOUT BASEBALL,TELL ME ABOUT BASEBALL,What is Baseball,1.000000
540,"you'll remember to do it, but you won't have t...","you'll remember to do it, but you won't have t...","okay, i'll take it out front right now.",1.000000
541,you are not smart,you are not smart,you may be right.,1.000000


In [56]:
predictions.head(10)

,Question,Prediction,Answer,Score
0,"i'll give you a speech like that, too.","i'll give you a speech like that, too.",do you think anyone will come to my funeral?,1.000000
1,"i know, you're absolutely right.","i know, you're absolutely right.",i wish it would cool off one day.,1.000000
2,i liked it.,i liked it.,"i'll give you a speech like that, too.",1.000000
3,the baby was eight pounds six ounces.,the baby was 8 pounds 6 ounces.,that's good to hear.,0.852941
4,I was sold a wireless service unavailable in m...,I was sold a wireless service unavailable in m...,"I see it here,we charged you $5 extra a month.",1.000000
5,maybe four or five pounds?,maybe four or five pounds?,my waist is bigger than it was.,1.000000
6,do you know hal,do you know hal,hal is the famous artificial intelligence fro...,1.000000
7,"Yeah,actually a lot of them are.","Yeah,actually a lot of them are.",How does she act?,1.000000
8,"if it's old age, why don't both of your hands ...","if it's old age, why don't both of your hands ...",that's a good question. maybe it's not old age.,1.000000
9,it's supposed to start at about eight.,it's supposed to start at about eight.,how many invitations has she given out?,1.000000


In [73]:
predictions.head(50)

,Question,Prediction,Answer,Score
0,"i'll give you a speech like that, too.","i'll give you a speech like that, too.",do you think anyone will come to my funeral?,1.000000
1,"i know, you're absolutely right.","i know, you're absolutely right.",i wish it would cool off one day.,1.000000
2,i liked it.,i liked it.,"i'll give you a speech like that, too.",1.000000
3,the baby was eight pounds six ounces.,the baby was 8 pounds 6 ounces.,that's good to hear.,0.852941
4,I was sold a wireless service unavailable in m...,I was sold a wireless service unavailable in m...,"I see it here,we charged you $5 extra a month.",1.000000
5,maybe four or five pounds?,maybe four or five pounds?,my waist is bigger than it was.,1.000000
6,do you know hal,do you know hal,hal is the famous artificial intelligence fro...,1.000000
7,"Yeah,actually a lot of them are.","Yeah,actually a lot of them are.",How does she act?,1.000000
8,"if it's old age, why don't both of your hands ...","if it's old age, why don't both of your hands ...",that's a good question. maybe it's not old age.,1.000000
9,it's supposed to start at about eight.,it's supposed to start at about eight.,how many invitations has she given out?,1.000000


In [57]:
results = predictions['Answer']

In [59]:
len(results)

543

In [60]:
test_df.shape

(543, 2)

In [61]:
test_df.columns

Index(['I.D.', 'Question'], dtype='object')

In [62]:
submission1 = pd.DataFrame(columns=['I.D.','Answer'])

In [64]:
submission1['I.D.'] = test_df['I.D.']
submission1['Answer'] = predictions['Answer']

In [65]:
submission1.shape

(543, 2)

In [66]:
submission1.head()

,I.D.,Answer
0,QN_1,do you think anyone will come to my funeral?
1,QN_2,i wish it would cool off one day.
2,QN_3,"i'll give you a speech like that, too."
3,QN_4,that's good to hear.
4,QN_5,"I see it here,we charged you $5 extra a month."


In [68]:
submission1.to_csv('SushmaD_1.csv',index=False)

In [78]:
not_right = predictions[(predictions['Score'] < 1) | (predictions['Score'] > 1)]

In [79]:
len(not_right)

51

In [81]:
predictions.to_csv('results.csv',index=False)